In [ ]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import regularizers
import numpy as np
import os

# Solicitar el archivo txt al usuario
from google.colab import files

print("Por favor, sube el archivo .txt con las intervenciones del parlamentario.")
uploaded = files.upload()

# Leer el archivo .txt
filename = list(uploaded.keys())[0]
with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocesamiento del texto
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = text.replace('\n', ' ')  # Reemplazar saltos de línea por espacios
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    return text

text = preprocess_text(text)
words = text.split()  # Dividir el texto en palabras únicas
vocab = sorted(set(words))  # Crear vocabulario único

# Crear diccionarios de mapeo
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# Convertir el texto a índices
encoded_text = [word_to_idx[word] for word in words]

# Preparar los datos para el modelo
sequence_length = 15  # Ajustar longitud de las secuencias
X = []
y = []

for i in range(len(encoded_text) - sequence_length):
    X.append(encoded_text[i:i+sequence_length])
    y.append(encoded_text[i+sequence_length])

X = np.array(X)
y = np.array(y)

# Construir el modelo LSTM equilibrado
vocab_size = len(vocab)
embedding_dim = 128  # Mantener dimensiones de embedding moderadas
lstm_units = 512  # Número moderado de unidades LSTM

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=sequence_length),
    LSTM(lstm_units, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.2),  # Dropout moderado
    LSTM(lstm_units, return_sequences=False, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.2),
    Dense(vocab_size, activation='softmax')
])

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Función para programar el ajuste de la tasa de aprendizaje
def scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)  # Asegurarse de que el valor devuelto sea un float
    else:
        new_lr = lr * tf.math.exp(-0.1)  # Ajustar la tasa de aprendizaje
        return float(new_lr)  # Asegurarse de que el valor devuelto sea un float

# Añadir callback para guardar el modelo con el mejor rendimiento
checkpoint = ModelCheckpoint('best_model.keras', monitor='loss', save_best_only=True)

# Añadir Learning Rate Scheduler
lr_scheduler = LearningRateScheduler(scheduler)

# Entrenamiento moderado, con más épocas pero controlado
print("Entrenando el modelo... Esto puede tomar varias horas.")
model.fit(X, y, epochs=70, batch_size=256, verbose=1, callbacks=[checkpoint, lr_scheduler])

# Función para generar texto con temperatura
def generate_text(model, start_sequence, max_words=200, temperature=0.7):
    input_sequence = [word_to_idx[word] for word in start_sequence.split() if word in word_to_idx]
    generated = start_sequence

    for _ in range(max_words):
        input_padded = tf.keras.preprocessing.sequence.pad_sequences([input_sequence], maxlen=sequence_length)
        predictions = model.predict(input_padded, verbose=0).flatten()
        predictions = np.log(predictions + 1e-9) / temperature
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        predicted_idx = np.random.choice(len(probabilities), p=probabilities)
        predicted_word = idx_to_word[predicted_idx]

        generated += ' ' + predicted_word
        input_sequence.append(predicted_idx)
        input_sequence = input_sequence[1:]

        # Finalizar si se alcanza un final lógico
        if predicted_word in ['.', '!', '?']:
            break

    return generated

# Interactuar con el modelo sin preguntar la longitud
print("¡Puedes comenzar a interactuar con el modelo!")
while True:
    prompt = input("Introduce el inicio del texto (o escribe 'salir' para terminar): ")
    if prompt.lower() == 'salir':
        print("¡Hasta luego!")
        break
    print("Respondiendo...")
    print(generate_text(model, prompt, max_words=200, temperature=0.7))


Por favor, sube el archivo .txt con las intervenciones del parlamentario.


Saving intervencionesCasado.txt to intervencionesCasado (1).txt
Entrenando el modelo... Esto puede tomar varias horas.
Epoch 1/70
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.0379 - loss: 11.8290 - learning_rate: 0.0010
Epoch 2/70
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0451 - loss: 6.2852 - learning_rate: 0.0010
Epoch 3/70
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0431 - loss: 6.2672 - learning_rate: 0.0010
Epoch 4/70
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0484 - loss: 6.2718 - learning_rate: 0.0010
Epoch 5/70
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0469 - loss: 6.2860 - learning_rate: 0.0010
Epoch 6/70
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0488 - loss: 6.2464 - learning_rate: 0.0010
Epoch 7/70
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0464 - loss: 6.2408 - learning_rate: 0.0010
Epoch 8/70
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0473 - loss: 6.2635 - learning_rate: 0.0010
Epoch 9/